# Выделение ключевых слов

__Идея:__ я хочу проверить, насколько хорошо будет работать матрица совстречаемости, которая будет основана на полном проходе по всему тексту (а не случайному выбору, пусть и большое количество раз, как в random walk, который мы обсуждали на парах).

_Дополнительная идея:_ посмотреть, как результат будет зависеть от величины «окна», которую мы зададим.

## Алгоритм

### Обработка текста

In [1]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [2]:
mystem = Mystem()
stops = stopwords.words("russian")
punctuators = punctuation + "«–»— \r\n"

In [3]:
def lemmatize_text(text):
    """Из исходного текста делает список лемм, очищенных от пунктуации
    и стоп-слов.
    
    :arg text (str): исходный текст, как он загружен из файла
    :returns clean_text (list of str): список лемм текста в том порядке, в
    котором они были в тексте
    """
    lemmas_raw = [parse["analysis"][0]["lex"] for parse in mystem.analyze(text)
                  if parse.get("analysis")]
    clean_text = [lemma_raw for lemma_raw in lemmas_raw
                 if lemma_raw not in stops and lemma_raw not in punctuators]        
    return clean_text

### Составление матрицы смежности

In [5]:
import numpy as np

In [24]:
def fill_co_occurence_matrix(text_lemmas, word_list, window):
    """Собирает матрицу совстречаемости из текста.
    
    :arg text_lemmas (list of str): лемматизированный и почищенный текст
    :arg word_list (list of str): список уникальных слов
    :arg window (int): окно, в котором мы считаем совстречаемость
    
    :returns M_co_occur (np.ndarray): матрица совстречаемости
    """
    M_co_occur = np.zeros((len(word_list), len(word_list)))
    for lemma_ind, lemma in enumerate(text_lemmas):
        lemma_matrix_index = word_list.index(lemma)
        ind_left = max(0, lemma_ind - window)
        ind_right = min(lemma_ind + window, len(text_lemmas))
        for ind_neighbor in range(ind_left, ind_right):
            word_neighbor = text_lemmas[ind_neighbor]
            neighbor_matrix_index = word_list.index(word_neighbor)
            M_co_occur[lemma_matrix_index][neighbor_matrix_index] += 1
    return M_co_occur

Предположим, что ключевыми будут те слова, которые соединяются с наибольшим количеством других слов => их совстречаемость будет самой высокой.

In [7]:
def extract_keywords_from_matrix(matrix, word_list, threshold):
    """Из матрицы совстречаемости выбирает сколько-то слов с наибольшей 
    совстречаемостью.
    
    :arg matrix (np.ndarray): матрица совстречаемости
    :arg word_list (list of str): список уникальных слов, на котором строилась 
    матрица
    :arg threshold (int): сколько первых значений совстречаемости брать
    
    :returns keywords (list of str): сами ключевые слова
    """
    kw_indices = np.where(np.argmax(matrix, axis=1) <= threshold)[0]
    keywords = [word_list[kw_ind] for kw_ind in kw_indices]
    return keywords

### Измеряем качество

In [40]:
def precision_recall_fmeasure (manual, predicted):
    '''считает точность, полноту и F-меру'''
    precisions = []
    recalls = []

    for index, words_manual in enumerate (manual):
        words_predicted = predicted [index]
        intersection = len (set(words_manual) & set (words_predicted)) #  число элементов в пересечении списков
        recalls.append (intersection/len(words_manual)) 
        precisions.append (intersection/len(words_predicted))
        
    mean_precision = sum(precisions)/ len (precisions)
    mean_recall = sum(recalls)/ len (recalls)
    fmeasure =  ((2*mean_recall*mean_precision)/(mean_recall+mean_precision))
    return '\tТочность: {}, полнота: {}, F-мера {}'.format (mean_precision, mean_recall, fmeasure)

### Всё вместе

In [36]:
def pipeline_for_texts(text, kw_true):
    text_lemmas = lemmatize_text(text)
    word_list = list(set(text_lemmas))
    for window in range(1, 6):
        print("window={}".format(window))
        M = fill_co_occurence_matrix(text_lemmas, word_list, window)
        for thresh in range(1, 4):
            kw = extract_keywords_from_matrix(M, word_list, thresh)
            print("\ttop-{} co-occurences: {}".format(thresh, kw))
            print(precision_recall_fmeasure(kw_true, kw))

## Подгружаем данные из датасета

Датасет — [ru-kw-eval](https://github.com/mannefedov/ru_kw_eval_datasets), я взяла одну из Независимых газет (ха-ха).

In [12]:
import json

In [13]:
ng_data = []
with open("../data/ng_0.jsonlines", "r") as f:
    for line in f.readlines():
        ng_data.append(json.loads(line))

Возьмём какую-нибудь случайную статью и посмотрим, как работает:

In [45]:
import random

In [48]:
random.seed() 
article_ind = choice(range(0, len(ng_data)-1))
article_data = ng_data[article_ind]
print(article_data["title"].upper())
pipeline_for_texts(article_data["content"], article_data["keywords"])

ОППОЗИЦИЯ ГРУЗИИ УГРОЖАЕТ ВСЕНАРОДНЫМ ВОССТАНИЕМ
window=1
	top-1 co-occurences: ['таргамадзе', 'май', 'напоминать', 'схватка', 'становиться']
	Точность: 0.2928571428571428, полнота: 0.3194444444444444, F-мера 0.30557355800388847
	top-2 co-occurences: ['таргамадзе', 'май', 'рейтинг', 'придавать', 'напоминать', 'схватка', 'начало', 'становиться']
	Точность: 0.22182539682539681, полнота: 0.2638888888888889, F-мера 0.24103576615831518
	top-3 co-occurences: ['таргамадзе', 'май', 'рейтинг', 'искусственный', 'придавать', 'напоминать', 'схватка', 'нагнетание', 'начало', 'становиться']
	Точность: 0.2260989010989011, полнота: 0.3194444444444444, F-мера 0.2647856982990152
window=2
	top-1 co-occurences: ['таргамадзе', 'май', 'приближение', 'напоминать', 'схватка', 'ясно', 'телеканал', 'георгий', 'становиться']
	Точность: 0.24545454545454545, полнота: 0.3194444444444444, F-мера 0.27760393383996423
	top-2 co-occurences: ['таргамадзе', 'май', 'рейтинг', 'придавать', 'чей', 'приближение', 'напоминать'